# When Rotten Tomatoes Isn't Enough: Twitter Sentiment Analysis with DSE

#### A demo using DataStax Enterprise Analytics, Apache Cassandra, Apache Spark, python, Jupyter Notebooks, twitter api, pattern, and sentiment analysis

### Things To Setup
* Create a Twitter Account and get API access: https://developer.twitter.com/en/docs/ads/general/guides/getting-started.html
* Install DSE https://docs.datastax.com/en/install/doc/install60/installTOC.html
* Start DSE Analytics Cluster: dse cassandra -k #Must use -k option for Analytics
* Set and Source Twitter enviroment variables in shell you will start Jupyter from
* CONSUMER_KEY 
* CONSUMER_SECRET 
* ACCESS_TOKEN 
* ACCESS_TOKEN_SECRET
* Install Anaconda and Jupyter #Anaconda is not required but will make installing jupyter easier 
* Find full path to <>/dse-6.0.1/resources/spark/python/lib/pyspark.zip
* Find full path to <>/dse-6.0.1/resources/spark/python/lib/py4j-0.10.4-src.zip
* Start Jupyter with DSE to get all environemnt variables: dse exec jupyter notebook
* !pip install cassandra-driver
* !pip install tweepy 
* !pip install pattern 
* !pip install panadas
* Counter-intuitive don't install pyspark!!

#### Add some environment variables to find dse verision of pyspark. Edit these varibles with your path.

In [281]:
pysparkzip = "/Users/amanda.moran/cassandra/dse-6.0.1/resources/spark/python/lib/pyspark.zip"
py4jzip = "/Users/amanda.moran/cassandra/dse-6.0.1/resources/spark/python/lib/py4j-0.10.4-src.zip"

In [280]:
# Needed to be able to find pyspark libaries
import sys
sys.path.append(pysparkzip)
sys.path.append(py4jzip)

#### Import python packages -- all are required

In [264]:
import pandas
import cassandra
import pyspark
import tweepy
import re
import os
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pattern.en import sentiment, positive

#### Helper function to have nicer formatting of Spark DataFrames

In [265]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

### Creating Tables, Pulling Tweets, and Loading Tables

#### Connect to DSE Analytics Cluster

In [266]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1']) #If you have a locally installed DSE cluster
session = cluster.connect()

#### Create Demo Keyspace 

In [267]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS dseanalyticsdemo 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set keyspace 

In [268]:
session.set_keyspace('dseanalyticsdemo')

#### Set Movie Title variable --Change this to search for different movies!

In [269]:
movieTitle = "missionimpossible"

In [270]:
positiveNegative = ["pos", "sad"] 

#### Create two tables in Cassandra for the movie title. One of negative tweets and one for positive tweets. Twitter returns a lot of information with each call but for this demo we will just utilize the twitter id (as our Primary key as it is unique) and the actual tweet. 
#### Is using twitter id the right value to distriubte by? Consider your data model when choosing your primary key. 

In [271]:
for emotion in positiveNegative: 
    
    query = "CREATE TABLE IF NOT EXISTS movie_tweets_%s_%s (twitterid bigint, tweet text, PRIMARY KEY (twitterid))" % (movieTitle, emotion)
    print query
    session.execute(query)


CREATE TABLE IF NOT EXISTS movie_tweets_missionimpossible_pos (twitterid bigint, tweet text, PRIMARY KEY (twitterid))
CREATE TABLE IF NOT EXISTS movie_tweets_missionimpossible_sad (twitterid bigint, tweet text, PRIMARY KEY (twitterid))


#### Setting up Search Terms for gathering tweets from Twitters API. The happy :) and sad :( face are twitter operators to find positive and negative tweets

In [272]:
searchTermSad= movieTitle + " :("
searchTermPos= movieTitle + " :)"

searchTerms = [searchTermSad, searchTermPos]

#### Function to CleanUp Each Tweet before if is inserted into Cassandra.
#### Removing: 
* emojis 
* flags 
* special characters 
* URL's 
* RT (for Retweet)

In [273]:
#Code from: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-

def cleanUpTweet(tweet):
    
    emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"
    u"(\ud83c[\udf00-\uffff])|"  
    u"(\ud83d[\u0000-\uddff])|" 
    u"(\ud83d[\ude80-\udeff])|"  
    u"(\ud83c[\udde0-\uddff])" 
    "+", flags=re.UNICODE)

    removeSpecial = re.compile ('[\n|#|@|!|.|?|,|\"]')
    removeHttp = re.compile("http\S+ | https\S+")
    removeRetweet = re.compile("RT")
    
    noemoji = emoji_pattern.sub(r'', tweet)
    nospecial = removeSpecial.sub(r'', noemoji)
    nohttp = removeHttp.sub(r'', nospecial)
    noretweet = removeRetweet.sub(r'', nohttp)
    
    cleanTweet=noretweet
    
    return cleanTweet

#### Required from Twitter: 
* consumer_key= ''
* consumer_secret= ''
* access_token=''
* access_token_secret=''

In [274]:

consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']

access_token = os.environ['ACCESS_TOKEN']
access_token_secret = os.environ['ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#### This cell will pull tweets from Twitter. The max number of tweets returned for free at one time is 100. 
#### Run this code a couple of times to get more data! 
#### Once the tweets are collected, loop over the list, clean up each tweet, and then insert it into the table. A large for loops surrounds this to make one call for postive tweets and one call for negative tweets. Happy and sad face have been URL encoded. :) = "%20%3A%29" and :( = "%20%3A%28"

In [275]:
for emotion in positiveNegative:
    print emotion
    public_tweets = 0
    query = "INSERT INTO movie_tweets_%s_%s (twitterid, tweet)" % (movieTitle, emotion)
    query = query + " VALUES (%s, %s)"
    
    if emotion == "pos":
        searchTermPos= movieTitle + "%20%3A%29"
        public_tweets = api.search(q=searchTermPos, lang="en", count="100")
    if emotion == "sad":
        searchTermSad= movieTitle + "%20%3A%28"
        public_tweets = api.search(q=searchTermSad, lang="en", count="100")

    for tweet in public_tweets:
        cleanTweet = cleanUpTweet(tweet.text)
        session.execute(query, (tweet.id, cleanTweet))
        print(cleanTweet)

pos
 TomCruise: Thank you Paris What an amazing time at the World Premiere of MissionImpossible Fallout
 TomCruise: Thank you Seoul for such a warm welcome MissionImpossible Fallout
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 THR: 'MissionImpossible – Fallout' is the first of Tom Cruise's movies to earn an A since 'The Last Samurai' 15 years ago
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 TroyCornejo: MissionImpossible MissionFilm Mission Impossible: Fallout is so damn good I highly recommend it I think Tom Cruise mig…
 MissionFilm: Your mission: g

 TomCruise: Thank you Paris What an amazing time at the World Premiere of MissionImpossible Fallout
 TomCruise: Thank you Seoul for such a warm welcome MissionImpossible Fallout
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 THR: 'MissionImpossible – Fallout' is the first of Tom Cruise's movies to earn an A since 'The Last Samurai' 15 years ago
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
 TroyCornejo: MissionImpossible MissionFilm Mission Impossible: Fallout is so damn good I highly recommend it I think Tom Cruise mig…
 MissionFilm: Your mission: get t

#### Do a select * on each table and verify that the tweets have been inserted into each Cassandra table

In [276]:
for emotion in positiveNegative:
    print emotion
    query = 'SELECT * FROM movie_tweets_%s_%s limit 10' % (movieTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print (user_row.twitterid, user_row.tweet)

pos
(1022972192571760641, u" THR: Wtih the 'MissionImpossible \u2013 Fallout' stunt Tom Cruise became the first actor to perform a HALO parachute jump a fall of some\u2026")
(1022868257135583233, u' TomCruise: Today MissionImpossible Fallout')
(1022962924485140480, u' drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu\u2019s giant account is not This is ElectionInterferen\u2026')
(1022971829328269312, u' AmaTellYouTruth: Truth isn\u2019t impossible it\u2019s the BCCRSS\u2014it\u2019s illegal to use a legal name Click the link &gt;&gt;https://tco/KmC3aP5HYB\u2026')
(1022978478663069696, u' HenryCavillNews: \u201cI\u2019m gonna focus on making Superman the best Superman possible\u201d More of Henry Cavill on despiertamerica where he a\u2026')
(1022868321937444864, u'Best MissionImpossible yet TomCruise and the cast did a marvelous work Thank you chrismcquarrie')
(1023964046117298176, u' THR: Tom Cruise scored a major victory at a time when movie stars have 

### Finally time for Apache Spark! 

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [277]:
countTokens = udf(lambda words: len(words), IntegerType())

spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

tableNamePos = "movie_tweets_%s_pos" % (movieTitle.lower())
tableNameSad = "movie_tweets_%s_sad" % (movieTitle.lower())
tablepos = spark.read.format("org.apache.spark.sql.cassandra").options(table=tableNamePos, keyspace="dseanalyticsdemo").load()
tablesad = spark.read.format("org.apache.spark.sql.cassandra").options(table=tableNameSad, keyspace="dseanalyticsdemo").load()

print "Postive Table Count: "
print tablepos.count()
print "Negative Table Count: "
print tablesad.count()


Postive Table Count: 
1363
Negative Table Count: 
1364


#### Use Tokenizer to break up the sentences into indiviudals words

In [278]:
tokenizerPos = Tokenizer(inputCol="tweet", outputCol="tweetwords")
tokenizedPos = tokenizerPos.transform(tablepos)

dfPos = tokenizedPos.select("tweet", "tweetwords").withColumn("tokens", countTokens(col("tweetwords")))

showDF(dfPos)

tokenizerSad = Tokenizer(inputCol="tweet", outputCol="tweetwords")
tokenizedSad = tokenizerSad.transform(tablesad)

dfSad = tokenizedSad.select("tweet", "tweetwords").withColumn("tokens", countTokens(col("tweetwords")))

showDF(dfSad)

,tweet,tweetwords,tokens
0,MissionFilm: Are you in or out Accept the mis...,"[, missionfilm:, are, you, in, or, out, accept...",22
1,RottenTomatoes: MissionImpossible just keeps ...,"[, rottentomatoes:, missionimpossible just, ke...",14
2,TomCruise: Today MissionImpossible Fallout,"[, tomcruise:, today, missionimpossible, fallout]",5
3,realmonaghan: Happy MissionImpossible Fallout...,"[, realmonaghan:, happy, missionimpossible, fa...",22
4,Longest 15 minutes ever MissionImpossibleFallo...,"[longest, 15, minutes, ever, missionimpossible...",7


,tweet,tweetwords,tokens
0,MissionFilm: Are you in or out Accept the mis...,"[, missionfilm:, are, you, in, or, out, accept...",22
1,RottenTomatoes: MissionImpossible just keeps ...,"[, rottentomatoes:, missionimpossible just, ke...",14
2,TomCruise: Today MissionImpossible Fallout,"[, tomcruise:, today, missionimpossible, fallout]",5
3,realmonaghan: Happy MissionImpossible Fallout...,"[, realmonaghan:, happy, missionimpossible, fa...",22
4,Longest 15 minutes ever MissionImpossibleFallo...,"[longest, 15, minutes, ever, missionimpossible...",7


#### Using StopWordsRemover to remove all stop words. Interesting to see, people don't use many stop words with twitter!

In [257]:
removerPos = StopWordsRemover(inputCol="tweetwords", outputCol="tweetnostopwords")
removedPos = removerPos.transform(dfPos)

dfPosStop = removedPos.select("tweet", "tweetwords", "tweetnostopwords").withColumn("tokens", countTokens(col("tweetwords"))).withColumn("notokens", countTokens(col("tweetnostopwords")))

showDF(dfPosStop)

removerSad = StopWordsRemover(inputCol="tweetwords", outputCol="tweetnostopwords")
removedSad = removerSad.transform(dfSad)

dfSadStop = removedSad.select("tweet", "tweetwords", "tweetnostopwords").withColumn("tokens", countTokens(col("tweetwords"))).withColumn("notokens", countTokens(col("tweetnostopwords")))

showDF(dfSadStop)

,tweet,tweetwords,tweetnostopwords,tokens,notokens
0,MissionFilm: Are you in or out Accept the mis...,"[, missionfilm:, are, you, in, or, out, accept...","[, missionfilm:, accept, mission, get, tickets...",22,12
1,RottenTomatoes: MissionImpossible just keeps ...,"[, rottentomatoes:, missionimpossible just, ke...","[, rottentomatoes:, missionimpossible just, ke...",14,9
2,TomCruise: Today MissionImpossible Fallout,"[, tomcruise:, today, missionimpossible, fallout]","[, tomcruise:, today, missionimpossible, fallout]",5,5
3,realmonaghan: Happy MissionImpossible Fallout...,"[, realmonaghan:, happy, missionimpossible, fa...","[, realmonaghan:, happy, missionimpossible, fa...",22,16
4,Longest 15 minutes ever MissionImpossibleFallo...,"[longest, 15, minutes, ever, missionimpossible...","[longest, 15, minutes, ever, missionimpossible...",7,7


,tweet,tweetwords,tweetnostopwords,tokens,notokens
0,THR: Wtih the 'MissionImpossible – Fallout' s...,"[, thr:, wtih, the, 'missionimpossible, –, fal...","[, thr:, wtih, 'missionimpossible, –, fallout'...",24,18
1,TomCruise: Today MissionImpossible Fallout,"[, tomcruise:, today, missionimpossible, fallout]","[, tomcruise:, today, missionimpossible, fallout]",5,5
2,drwright4congr1: Why in the world is there a ...,"[, drwright4congr1:, why, in, the, world, is, ...","[, drwright4congr1:, world, ban, little, accou...",23,11
3,AmaTellYouTruth: Truth isn’t impossible it’s ...,"[, amatellyoutruth:, truth, isn’t, impossible,...","[, amatellyoutruth:, truth, isn’t, impossible,...",18,14
4,HenryCavillNews: “I’m gonna focus on making S...,"[, henrycavillnews:, “i’m, gonna, focus, on, m...","[, henrycavillnews:, “i’m, gonna, focus, makin...",21,14


### Sentiment Analysis using Python package Pattern

#### Convert each Spark Dataframe to a Pandas Dataframe. This works as-is because we are working with a small dataset. For larger datasets only convert to Pandas if data can fit in memory. From there loop over each row and get the sentiment score (anything + is postive and anything - or 0 is negative). The "positive" function will return true if the tweet is postive. For more info on how the scores are calcuated: https://www.clips.uantwerpen.be/pages/pattern-en#sentiment

#### Negative Tweets

In [258]:
pandaSad = dfSadStop.toPandas()
movieScoreSad = 0
countSad = 0
numSadTweets = 0

for index, row in pandaSad.iterrows():
    print row['tweet']
    print sentiment(row["tweetnostopwords"])
    print positive(row["tweetnostopwords"], .1)
    if positive(row["tweetnostopwords"], .1):
        print "This is a negative tweet! Analysis is wrong :("
        countSad = countSad + 1
    scoreSad = sentiment(row['tweetnostopwords'])[0]
    if scoreSad <= 0:
        movieScoreSad = scoreSad + movieScoreSad

 MyFilmClub: To celebrate the release of Mission: Impossible – Fallout in cinemas now (cert 12A) we are offering you the chance to win…
(-0.6666666666666666, 1.0)
False
 pshanley88: Where does the MissionImpossible series rank on the list of greatest action franchises of all time We take a look on this…
(0.09999999999999998, 0.6666666666666666)
False
TomCruise  How the hell is he 56  And those stunts MissionImpossible Fallout  a must watch for all Action…
(0.0, 0.0)
False
 observer: The RottenTomatoes 97% score for MissionImpossible is the highest of any in the series Projections hint at a $50 million o…
(0.0, 0.0)
False
 HenryCavillNews: New picture of Henry Cavill doing interviews in Japan: “A gentleman who gives a chair to the lady” via movieplusjp…
(0.13636363636363635, 0.45454545454545453)
True
This is a negative tweet! Analysis is wrong :(
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
 WilliamBibbiani:  if you want Thandie Newton to come back to the MissionImpossib

 MtnMD: What were they planning on doing with these stolen children SELLING THEM We need a …
(0.0, 0.0)
False
 drose: Really enjoyed an advanced screening of MissionFilm Ya’ll go see it when it opens July 27 Best one yet MissionImpossible ht…
(0.6333333333333333, 0.5333333333333333)
True
This is a negative tweet! Analysis is wrong :(
 realmonaghan: Right Everytime I see it my mind is blown  missionimpossible
(0.2857142857142857, 0.5357142857142857)
True
This is a negative tweet! Analysis is wrong :(
SEMrush data shows that search volumes for MissionImpossible movies are constantly growing by 1M from film to film…
(0.0, 0.3333333333333333)
False
 demelzac: BetyX1 mametom1269 joelledray22 Crazy4TomCruise TomsCruiser LolyCruise wjy7k1 Becca902450 Lovely49200223 NaR5pm Kg…
(0.0, 0.0)
False
 RottenTomatoes: TomCruise's MissionImpossible Fallout is the highest rated film in the franchise - it's CertifiedFresh at 97% with…
(0.0, 0.0)
False
 realmonaghan: Happy MissionImpossible Fallout Day I’

MissionImpossible hello world
(0.0, 0.0)
False
 MissionFilm: Certified Fresh and ready to rumble Don’t miss MissionImpossible Fallout starring Tom Cruise In theatres now
(0.25, 0.5)
True
This is a negative tweet! Analysis is wrong :(
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
(0.0, 1.0)
False
 mysantikos: Can you handle all the action coming up this weekend at MySantikos MissionImpossible Fallout and TeenTitansGo To the Mo…
(0.1, 0.1)
True
This is a negative tweet! Analysis is wrong :(
 studiobinder: Shooting a fight scene Use our shot list from Mission Impossible Fallout to kick things off: Mi…
(-0.6666666666666666, 1.0)
False
 iamVikramPrabhu: A huge fan of the MissionImpossible franchise MissionImpossibleFallout just WOW TomCruise 81-18 Keep running…
(0.25000000000000006, 0.95)
True
This is a negative tweet! Analysis is wrong :(
Saw the Mission Imposible film this afternoon absolutely action pa

Counting down until tonight’s screening of MissionImpossible Fallout Popped on Rogue Nation and wow- what a stunt…
(0.0, 0.0)
False
 SonofLiberty357: MissionImpossible Don’t worry nobody is accepting THAT mission ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
(0.0, 0.0)
False
Some Missions Are Not a Choice Mission: Impossible - Fallout is now playing Check our website for showtimes &amp; to…
(-0.6666666666666666, 1.0)
False
 sweet_messenger: This review of MissionImpossible by KendraJames_ is SO good and totally captures what an amazing and fun movie it is…
(0.4, 0.6125)
True
This is a negative tweet! Analysis is wrong :(
 TheRoot: Watch: VeronicaWebb chats with ImAngelaBassett about her role in MissionImpossible Fallout how she chooses her roles as a…
(0.0, 0.0)
False
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
(0.0, 1.0)
False
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + openi

(0.0, 0.0)
False
 screenrant: MissionImpossible keeps getting betterFallout just scored the franchise's biggest box office debut
(0.0, 0.0)
False
Just putting it out there but after seeing Henry cavill in MissionImpossible he should so do a magnum PI movie Du…
(0.0, 0.0)
False
 MissionFilm: This mission is a go MissionImpossible
(0.0, 0.0)
False
 DJs_Corner: FridayFeeling EDM dancemusic fans MissionImpossible MissionFilm is a must see this weekend H…
(0.0, 0.0)
False
 RegalMovies: Excited to have Henry Cavill (and his dog Kal) stop by a Regal theatre and talk about his new MissionImpossible movie Ch…
(0.2556818181818182, 0.6022727272727273)
True
This is a negative tweet! Analysis is wrong :(
 steph93065: Dr Ken Wright is a citizen Republican Congressional candidate running against Ted Lieu  Guess which one is shadow banned…
(0.0, 0.0)
False
 MtnMD: What were they planning on doing with these stolen children SELLING THEM We need a …
(0.0, 0.0)
False
 MissionFilm: The fans have spoken an

 drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu’s giant account is not This is ElectionInterferen…
(-0.09375, 0.75)
False
 Variety: Ranking the MissionImpossible movies: Which one comes out on top
(0.5, 0.5)
True
This is a negative tweet! Analysis is wrong :(
 steph93065: Dr Ken Wright is a citizen Republican Congressional candidate running against Ted Lieu  Guess which one is shadow banned…
(0.0, 0.0)
False
 TomCruise: A full year in the making Check out how we did it MissionImpossible Fallouthttps://tco/FMsPWfSZfR
(0.35, 0.55)
True
This is a negative tweet! Analysis is wrong :(
 GiteshPandya: With last yr's Mummy &amp; American Made auds showed they won't always pay to see TomCruise But when in his wheelhouse lik…
(0.0, 0.03333333333333333)
False
 retroSFXsports: Happy MissionImpossible - Fallout day Remember the first MI movie Two years ago we re-dubbed it with RollingThunder…
(0.525, 0.6666666666666666)
True
This is a negative tweet! Analysis is w

 thehunter_kr: 180716 HQ Preview 헨리카빌 HenryCavill MissionImpossible
(0.0, 0.0)
False
 drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu’s giant account is not This is ElectionInterferen…
(-0.09375, 0.75)
False
 MissionFilm: The movie of the summer has arrived Don't miss MissionImpossible Fallout starring Tom Cruise Now playing in theatres
(0.0, 0.0)
False
 RottenTomatoes: Critics say MissionImpossible Fallout is not only the best of the series but possibly an all-time great
(0.6, 0.6833333333333332)
True
This is a negative tweet! Analysis is wrong :(
REVIEW: 'Mission Impossible: Fallout' is the Summer's Best Action Film MissionImpossible MissionImpossibleFallout
(0.55, 0.2)
True
This is a negative tweet! Analysis is wrong :(
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
(0.0, 1.0)
False
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST open

 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
(0.0, 1.0)
False
 drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu’s giant account is not This is ElectionInterferen…
(-0.09375, 0.75)
False
It's no secret that we have quite the collection of aerial spy equipment It’s reconnaissance aircraft equipped wi…
(-0.4, 0.7)
False
seeing how dial-up email and floppy drives/disks are presented as high tech in the first MissionImpossible movie i…
(0.20500000000000002, 0.43666666666666665)
True
This is a negative tweet! Analysis is wrong :(
 radiocityindia: ContestAlert Here's an opportunity to win couple movie tickets for MissionImpossibleQ Mission: Impossible – Fallou…
(0.06666666666666671, 0.7)
False
 iamVikramPrabhu: A huge fan of the MissionImpossible franchise MissionImpossibleFallout just WOW TomCruise 81-18 Keep running…
(0.25000000000000006, 0.95)
True
This is a negative twe

 MissionFilm: The movie of the summer has arrived Don't miss MissionImpossible Fallout starring Tom Cruise Now playing in theatres
(0.0, 0.0)
False
 MissionFilm: The fans have spoken and they can't get enough of the action-packed new mission See MissionImpossible Fallout in theatre…
(0.06818181818181818, 0.4772727272727273)
False
 MissionFilm: Certified Fresh and ready to rumble Don’t miss MissionImpossible Fallout starring Tom Cruise In theatres now
(0.25, 0.5)
True
This is a negative tweet! Analysis is wrong :(
 julianlytle: I’m late in posting this but here’s my MissionImpossible Fallout review on idobiradio
(-0.3, 0.6)
False
 PUBGMOBILE: Your MissionImpossible if you choose to accept it is to compete in a campaign to collect rare items Should you be succe…
(0.3, 0.9)
True
This is a negative tweet! Analysis is wrong :(
 iamVikramPrabhu: A huge fan of the MissionImpossible franchise MissionImpossibleFallout just WOW TomCruise 81-18 Keep running…
(0.25000000000000006, 0.95)
True
This 

#### Positive Tweet
#### Also adding up all the sentiment scores of all the tweets

In [259]:
pandaPos = dfPosStop.toPandas()
movieScore = 0
countPos = 0
for index, row in pandaPos.iterrows():
    print row['tweet']
    print sentiment(row["tweetnostopwords"])
    print positive(row["tweetnostopwords"])
    if not positive(row["tweetnostopwords"]) and sentiment(row["tweetnostopwords"])[0] != 0.0:
        print "This is a postive tweet! Analysis is wrong :("
        countPos = countPos + 1
    score = sentiment(row['tweetnostopwords'])[0]
    movieScore = score + movieScore



 THR: Wtih the 'MissionImpossible – Fallout' stunt Tom Cruise became the first actor to perform a HALO parachute jump a fall of some…
(0.25, 0.3333333333333333)
True
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
 drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu’s giant account is not This is ElectionInterferen…
(-0.09375, 0.75)
False
This is a postive tweet! Analysis is wrong :(
 AmaTellYouTruth: Truth isn’t impossible it’s the BCCRSS—it’s illegal to use a legal name Click the link &gt;&gt;https://tco/KmC3aP5HYB…
(-0.3222222222222222, 0.5666666666666667)
False
This is a postive tweet! Analysis is wrong :(
 HenryCavillNews: “I’m gonna focus on making Superman the best Superman possible” More of Henry Cavill on despiertamerica where he a…
(1.0, 0.3)
True
Best MissionImpossible yet TomCruise and the cast did a marvelous work Thank you chrismcquarrie
(1.0, 0.65)
True
 TalkFilmSoc: The Burj Khalifa sequence in GHOST PROTOCOL was the shock to t

 LightsCameraPod: Bless someone's timeline with this Henry Cavill MissionImpossible GIF
(0.0, 0.0)
False
 RottenTomatoes: Critics say MissionImpossible Fallout is not only the best of the series but possibly an all-time great
(0.6, 0.6833333333333332)
True
 MissionFilm: The fans have spoken and they can't get enough of the action-packed new mission See MissionImpossible Fallout in theatre…
(0.06818181818181818, 0.4772727272727273)
False
This is a postive tweet! Analysis is wrong :(
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
 dbapz: Favourite films of 2018 so far: 1 Mission: Impossible - Fallout2 Mission: Impossible - Fallout3 Mission: Impossible - Fal…
(-0.6666666666666666, 1.0)
False
This is a postive tweet! Analysis is wrong :(
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
WATCH: SimonPegg on returning as tech whiz Benji Dunn for MissionImpossible – Fallout MissionFilm…
(0.0, 0.0)
False
I got to rank the films in the MissionImpossible franchise which 

 MissionFilm: This mission is a go MissionImpossible
(0.0, 0.0)
False
 ThrillerMagazin: After watching Mission Impossible: Fallout in IMAX last night here is our spolier-free review of the latest installme…
(0.25, 0.48333333333333334)
True
 IMAX: Extreme stunts extreme sound extreme experience MissionImpossible Fallout is coming to IMAX tonight
(0.0062500000000000056, 0.85)
False
This is a postive tweet! Analysis is wrong :(
 MissionFilm: The movie of the summer has arrived Don't miss MissionImpossible Fallout starring Tom Cruise Now playing in theatres
(0.0, 0.0)
False
 drwright4congr1: Why in the world is there a ban MY little account while Ted Lieu’s giant account is not This is ElectionInterferen…
(-0.09375, 0.75)
False
This is a postive tweet! Analysis is wrong :(
 RottenTomatoes: Critics say MissionImpossible Fallout is not only the best of the series but possibly an all-time great
(0.6, 0.6833333333333332)
True
 iamnavamohan: MissionImpossible is the rare Hollywood franchise tha

 MissionFilm: Real Stunts Real Action Real Fun See the movie EW calls “wildly entertaining” MissionImpossible Fallout starring Tom…
(0.26666666666666666, 0.25000000000000006)
True
 BookEnoch: MissionImpossible Why is MSM NOT reporting this  realDonaldTrump POTUS GenFlynn mflynnJR prayingmedic julianasange…
(0.0, 0.0)
False
For more on spies in the skies (and the record-setting reconnaissance aircraft the SR-71 Blackbird) check out…
(0.0, 0.0)
False
We're right there with you JKCorden Can't wait to see MissionImpossible Fallout this weekend GRBEntertainment
(0.2857142857142857, 0.5357142857142857)
True
If you want to know how much of a MissionImpossible (MissionFilm) nerd I am: my 3 best friends and I had IMF I…
(0.6, 0.25)
True
 iamnavamohan: MissionImpossible is the rare Hollywood franchise that is getting better with every passing installment all thanks to s…
(0.3333333333333333, 0.5333333333333333)
True
 SightSoundmag: Thrill of the hunt: Tom Cruise and the MissionImpossible franchi

(0.0, 0.0)
False
 realmonaghan: Happy MissionImpossible Fallout Day I’m so excited for all you fans to finally see this epic thrill ride I’m tremendou…
(0.31875000000000003, 0.7875)
True
 MissionFilm: MissionImpossible Fallout takes the stunts to the next level You'll have to see to believe Get tickets
(0.0, 0.0)
False
 taran_adarsh: MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a MissionImpossibl…
(0.0, 1.0)
False
 JayneQPatriot: They said MAGA was MissionImpossible 🤔But WeThePeople are WINNING 🤩Our economy is BOOMING 🤑Unemployment is D…
(0.5, 0.75)
True
Mission Impossible Fallout is my fave MI film out the series It was just totally awesome MissionImpossibleFallout…
(0.16666666666666669, 1.0)
True
 realmonaghan: Happy MissionImpossible Fallout Day I’m so excited for all you fans to finally see this epic thrill ride I’m tremendou…
(0.31875000000000003, 0.7875)
True
 ItzTomcruise: Prepare for the biggest boldest stunts yet Get a sneak

Basically the MissionImpossible franchise is about Tom Cruise trying to prove to us that he's not a sissy You c…
(0.0, 0.0)
False
 MyFilmClub: To celebrate the release of Mission: Impossible – Fallout in cinemas now (cert 12A) we are offering you the chance to win…
(-0.6666666666666666, 1.0)
False
This is a postive tweet! Analysis is wrong :(
 WilliamBibbiani:  if you want Thandie Newton to come back to the MissionImpossible franchise and kick butt with Ilsa Faust
(0.0, 0.0)
False
 virohq: Powered by Viro Media: Viro is proud to announce the Paramount AR+ app with the Mission: Impossible FALLOUT AR Experience Avai…
(0.06666666666666671, 1.0)
False
This is a postive tweet! Analysis is wrong :(
 RottenTomatoes: Critics say MissionImpossible Fallout is not only the best of the series but possibly an all-time great
(0.6, 0.6833333333333332)
True
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
 screenjunkies: We shaved some very important questions for MissionImpossible star He

This is a postive tweet! Analysis is wrong :(
 TomCruise: Today MissionImpossible Fallout
(0.0, 0.0)
False
 RottenTomatoes: TomCruise's MissionImpossible Fallout is the highest rated film in the franchise - it's CertifiedFresh at 97% with…
(0.0, 0.0)
False
 iamVikramPrabhu: A huge fan of the MissionImpossible franchise MissionImpossibleFallout just WOW TomCruise 81-18 Keep running…
(0.25000000000000006, 0.95)
True
 MissionFilm: MissionImpossible Fallout takes the stunts to the next level You'll have to see to believe Get tickets
(0.0, 0.0)
False
 EmpireCinemas: The best intentions often come back to haunt you MissionImpossible  for your chance to WIN a merch pack  In Cinem…
(0.6, 0.2333333333333333)
True
 Variety: “Fallout” earned the highest preview gross of all the MissionImpossible films
(0.0, 0.0)
False
 LolyCruise: Happy Friday and happy MissionImpossible day ❤❤ demelzac Lovely49200223 szilviade_ mametom1269 Crazy4TomCruise TomsC…
(0.8, 1.0)
True
 TomCruise: Today MissionImpossibl

 ShortList: Henry Cavill on owning (and then losing) the world's most famous facial hair HenryCavill MissionIm…
(0.25, 0.5)
True
Starmaiden5 gofooji Mission: accomplished MissionImpossible
(0.2, 0.5)
True
 iamnavamohan: With every MissionImpossible movie Tom Cruise made us ask how the hell did he do those stunts Well in this installment…
(0.0, 0.0)
False
 Variety: MissionImpossible - Fallout cruising toward record $60 million opening
(0.0, 0.0)
False
 HenryCavillNews: “I’m gonna focus on making Superman the best Superman possible” More of Henry Cavill on despiertamerica where he a…
(1.0, 0.3)
True
MissionImpossibleFallout has a TERRIFIC opening weekend BIGGEST opening day + opening weekend for a…
(0.0, 1.0)
False
 THR: Tom Cruise himself described the 'MissionImpossible – Fallout' stunt as one of his most dangerous stunts yet
(-0.6, 0.9)
False
This is a postive tweet! Analysis is wrong :(
 thehunter_kr: 180716 HQ Preview 헨리카빌 HenryCavill MissionImpossible
(0.0, 0.0)
False
MissionFilm l

MissionImossibleFallout super undi I liked it One of the best action movies I ever seen Edge of the seat adren…
(0.5083333333333333, 0.46666666666666673)
True
Research ahead of Fallout this weekend NowWatching MissionImpossible GhostProtocol
(0.0, 0.0)
False
Van Life Pacific Beach California USA Photography &amp; Art for your home business or gift …
(0.0, 0.0)
False
 PUBGMOBILE: We are proud to announce that pubgmobile has teamed up with MissionImpossible Fallout for a limited time worldwide event…
(0.3642857142857143, 0.5714285714285714)
True
 SonofLiberty357: MissionImpossible Don’t worry nobody is accepting THAT mission ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
(0.0, 0.0)
False
 steph93065: Dr Ken Wright is a citizen Republican Congressional candidate running against Ted Lieu  Guess which one is shadow banned…
(0.0, 0.0)
False
 RottenTomatoes: MissionImpossible just keeps getting better which is your favorite film in the franchise
(0.5, 0.75)
True
MissionImpossible FridayFeeling Jerry Jones FlashbackFrida

### Alright! Should I see this movie???

In [260]:
posrating = movieScore/(dfPos.count() - countPos)

print "Postive Rating Average Score: " 
print posrating
print "Number of Tweets Classified Incorrectly:"
print countPos 
if dfSad.count() != 0:
    sadrating = movieScoreSad/(dfSad.count() - countSad)
else: 
    sadrating = 0
print "Negative Rating Average Score:"
print sadrating
print "Number of Tweets Classified Incorrectly:"
print countSad


if posrating > abs(sadrating):
    print "People like this movie!"
elif posrating == abs(sadrating):
    print "People are split on this movie! Take a risk!"
elif posrating < abs(sadrating):
    print "People do not like this movie!"


Postive Rating Average Score: 
0.103508726158
Number of Tweets Classified Incorrectly:
199
Negative Rating Average Score:
-0.0810201231203
Number of Tweets Classified Incorrectly:
473
People like this movie!
